##### 0. 스페셜 토큰 처리?

In [1]:
# 분할 x 단어
# not_tokenized = ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', 
#                 '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', 
#                 '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']

##### 1. 발화자 / 대화 분리해서 학습

In [6]:
import pandas as pd
from koeda import EasyDataAugmentation
import re
import pandas as pd
import os
import nlpaug.augmenter.word as naw

In [8]:
!pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 32.8 MB/s eta 0:00:00


In [10]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /data/ephemeral/home/nltk_data...


True

In [11]:
# 데이터 로드
df = pd.read_csv('/root/NLP/data/train.csv')
df_filtered = df[['fname', 'dialogue', 'summary', 'topic']]

# Augmenter 초기화 (Synonym Replacement)
augmenter = naw.SynonymAug(aug_src='wordnet', aug_min=1, aug_max=3)

# 증강 데이터를 저장할 리스트
augmented_rows = []

# 증강 함수 정의
def augment_text_data_with_nlpaug(text, repetition=3):
    """입력된 문장에 대해 nlpaug를 사용하여 데이터 증강"""
    return [augmenter.augment(text) for _ in range(repetition)]

# 화자 태그와 텍스트를 분리하는 함수
def split_speaker_and_text(dialogue):
    """화자와 대화 내용을 분리하여 리스트로 반환"""
    lines = dialogue.split('\n')
    speaker_text_pairs = []
    for line in lines:
        if ':' in line:
            speaker, text = line.split(':', 1)
            speaker_text_pairs.append((speaker.strip(), text.strip()))
    return speaker_text_pairs

# 화자 태그와 텍스트를 결합하는 함수
def join_speaker_and_text(speaker_text_pairs):
    """화자와 대화 내용을 결합하여 하나의 문자열로 반환"""
    return '\n'.join([f"{speaker}: {text}" for speaker, text in speaker_text_pairs])

# 데이터 증강 수행
for index, row in df_filtered.iterrows():
    original_text = row['dialogue']

    # 화자 태그와 대화 내용 분리
    speaker_text_pairs = split_speaker_and_text(original_text)

    # 대화 내용에 nlpaug 적용
    augmented_text_lists = [
        augment_text_data_with_nlpaug(text, repetition=3) for _, text in speaker_text_pairs
    ]

    # nlpaug 결과와 화자 태그 결합
    for augmented_texts in zip(*augmented_text_lists):
        new_speaker_text_pairs = [
            (speaker_text_pairs[i][0], augmented_text)
            for i, augmented_text in enumerate(augmented_texts)
        ]
        augmented_dialogue = join_speaker_and_text(new_speaker_text_pairs)

        # 증강 데이터를 새로운 행으로 추가
        augmented_rows.append({
            'fname': row['fname'],
            'dialogue': augmented_dialogue,
            'summary': row['summary'],
            'topic': row['topic']
        })

    # 진행 상황 출력
    if index % 100 == 0:
        print(f"Processed {index + 1} rows out of {len(df_filtered)}")

# 증강 데이터를 DataFrame으로 변환
augmented_data = pd.DataFrame(augmented_rows)

# 결과 저장
augmented_data.to_csv('/root/NLP/data/aug_train.csv', index=False)

NameError: name 'wordnet' is not defined

##### 2. df + augmented_data concat하기

##### 3. 인덱스 reset / fname 열 바꾸기(train_0 ~ train?)